In [1]:
import xlrd
from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize, word_tokenize
#### 1.  Create the segments from each video
import pandas as pd
import xlsxwriter
import os

#### The characterisation work
################################################################################
import nltk
import codecs
import pronto
import itertools
#import owlready
#from owlready import *

from nltk.corpus import stopwords, wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, PorterStemmer
import signal, ctypes
from pronto import Ontology
from pronto import *
import re

from pandas import DataFrame
import math
from difflib import SequenceMatcher
import sys
import collections
from collections import Counter
import statistics 
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
lemmatizer = WordNetLemmatizer()

###############################################################################
videosExcelFile='health domain text.xlsx'    
xls1= pd.ExcelFile(videosExcelFile)
#xls2= pd.ExcelFile('JavaProgrammingDomainSegments3MinutesEach.xlsx')

workbook = xlsxwriter.Workbook('health Videos To 6 sentences using Full Stop as Delimiter.xlsx')

# Get the sheets from both excel workbooks
presentationSheetNames=xls1.sheet_names
#javaSheetNames=xls2.sheet_names

# calculate howmany videos I can create by joining segmnet from both domain.I need 5 segments from each domain to create 30 minutes video
# no of segments from presentation domain is less than from Jave, so I will use it to claculate # videos to avoid out of range issue

noOfVideos=len(presentationSheetNames)
print('# of videos = ',noOfVideos)
#print(sheetNames)

### Get the segments with thier metadata for the first video
allArtificialVideos=[]
artificialVideo=[]
videoMetaData=[]
artificialVideoWithoutTime=[]
allArtificialVideosWithoutTime=[]
counterOfDomainVideo=0
df_idP=[]
df_topicP=[]
df_textP=[]
df_timeP=[]
df_Video=[]
for i in  range(noOfVideos):       
    presentationSheetIteration = pd.read_excel(videosExcelFile, sheet_name=presentationSheetNames[i])
    
       
    ####### Get the presentation segment
    df_idP=presentationSheetIteration['VideoId'].tolist()
    
            
    df_Video.append([presentationSheetIteration['VideoId'].tolist()[0],
                     presentationSheetIteration['Transcript Time'].tolist(),
                     presentationSheetIteration['Transcript Text'].tolist()])
    
    
    #df_topicP.append(presentationSheetIteration['Topic'])
    

for i in df_Video:
    print(i)
    print()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Main\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# of videos =  3
['1TBs2hPO0R4', [0.03, 0.04, 0.07, 0.09, 0.11, 0.14, 0.16, 0.19, 0.21, 0.22, 0.23, 0.25, 0.28, 0.3, 0.32], ['Making an investment in public', 'transportation will improve our quality', 'of life here in Central Texas,  but it', 'will also have a major impact on our', 'environment, our climate, our air and our', 'water. Well, been a leader on', 'climate, change and you know a', 'leader in renewable energy and energy', 'efficiency', 'far behind on dealing with', 'transportation problem. The project', 'Connect division is a big, bold one, and', 'going to require some change.', 'going to require investment and we need', 'to bring people together to support that']]

['0mnuGApw5Tw', [0.0, 0.02, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.2, 0.2, 0.22, 0.23, 0.26, 0.28, 0.3, 0.32, 0.34, 0.37, 0.39, 0.4, 0.42, 0.45, 0.46, 0.49, 0.5, 0.52, 0.54, 0.55, 0.56, 0.58, 0.59, 1.01, 1.02, 1.05, 1.07, 1.09, 1.1, 1.13, 1.15, 1.17, 1.19, 1.2, 1.22, 1.24, 1.26, 1.28, 1.3, 1.32, 1.34, 

In [2]:
allVideosSeg=[]
for v in df_Video:
    
    videoTime=v[1]
    videoTranscript=v[2]
    
    segTime=[]
    segText=[]
    segment=[]
    wordsCollected=[]
    start=0
    fullStopFound='False'
    videoSegemnts=[]

    for i in range(start,len(videoTime)):
            #print(videoTime[i],'    ',videoTime.index(videoTime[i]),'    ',videoTranscript[i].split())
            #print()



            #print('videoTranscript[i] ',videoTranscript[i])
            if str(videoTranscript[i]) !='nan':
                print('videoTranscript[i] ',videoTranscript[i])
                newLine=videoTranscript[i].split()# for each text line in the transcript, split it into list of words
                #print('newLine = ',newLine)
                #print()
                segText.append(videoTranscript[i])
                #print('segText = ',segText)

            #numberOfword=len(newLine)# count ho mqany word per line
            #print('start = ',start)
            for j in range(0, len(newLine)):

                if newLine[j][-1]!='.':
                    print('videoTime[i] =',videoTime[i])
                    if videoTime[i] not in segTime:
                        segTime.append(videoTime[i])
                        wordsCollected.append(newLine[j])

                else:
                    #print('segTime = ',segTime)
                    #print()
                    if segTime:
                        segment.append([segTime[0],segTime[-1],segText])
                        videoSegemnts.append([segTime[0],segTime[-1],segText])
                        #print('segment= ',segment)
                        #print()
                    segment=[]
                    segText=[]
                    segTime=[]
                    strat=videoTime[i]
                    #break
    
    if videoSegemnts:
        segm= videoSegemnts[-1] 
    segment=[]
    #print(videoTime.index(segm[1]))
    temptimeList=[]
    tempText=[]
    if segm[1] != videoTime[-1]:
        start=videoTime.index(segm[1])
        for i in range(start,len(videoTime)):
            temptimeList.append(videoTime[i])
            tempText.append(videoTranscript[videoTime.index(videoTime[i])])
    if len(temptimeList)>1:

        videoSegemnts.append([temptimeList[0],temptimeList[-1],tempText])
        
        
    elif len(temptimeList)==1:
        videoSegemnts.append([temptimeList[0],tempText])
        
    allVideosSeg.append([v[0],videoSegemnts])
    videoSegemnts=[]





                    
                
                

videoTranscript[i]  Making an investment in public
videoTime[i] = 0.03
videoTime[i] = 0.03
videoTime[i] = 0.03
videoTime[i] = 0.03
videoTime[i] = 0.03
videoTranscript[i]  transportation will improve our quality
videoTime[i] = 0.04
videoTime[i] = 0.04
videoTime[i] = 0.04
videoTime[i] = 0.04
videoTime[i] = 0.04
videoTranscript[i]  of life here in Central Texas,  but it
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTime[i] = 0.07
videoTranscript[i]  will also have a major impact on our
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTime[i] = 0.09
videoTranscript[i]  environment, our climate, our air and our
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTime[i] = 0.11
videoTranscript[i]  water. Well, been a leader on
v

In [3]:
print('# of videos = ',len(allVideosSeg))
print()
for i in allVideosSeg:
    for j in i[1]:
        print(j)
        print()
    print('********* END of Video ***************')
    print()


# of videos =  3

[0.03, 0.11, ['Making an investment in public', 'transportation will improve our quality', 'of life here in Central Texas,  but it', 'will also have a major impact on our', 'environment, our climate, our air and our', 'water. Well, been a leader on']]

[0.14, 0.23, ['climate, change and you know a', 'leader in renewable energy and energy', 'efficiency', 'far behind on dealing with', 'transportation problem. The project']]

[0.23, 0.28, ['Connect division is a big, bold one, and', 'going to require some change.']]

[0.28, 0.32, ['going to require some change.', 'going to require investment and we need', 'to bring people together to support that']]

********* END of Video ***************

[0.0, 0.07, ['going on everybody in this video,', 'going to be speaking about how to', 'make your environment a better place,', 'more comfortable work for you make you', 'feel better in your day life. It']]

[0.07, 0.09, ['will improve your quality of life. I']]

[0.09, 0.3, ['believe 

In [4]:
worksheet = workbook.add_worksheet('sheet2')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
row=1
col=0

for i in allVideosSeg:
    col=0
    count=1
    
    
    # start,end and transcript are saved as second element in i=1
    for j in i[1]:
        worksheet.write(row,col,str(i[0])+str(count))
        
        worksheet.write(row,col+1,j[0]) # write the start time
        worksheet.write(row,col+2,j[1]) # write the end time
        tempCol=col+3
        
        if j[2]:
            for k in j[2]:
                
                worksheet.write(row,tempCol,k)
               
                row+=1
        else:
            worksheet.write(row,tempCol,'')
            row+=1
        count+=1
    
      
workbook.close()
   